#### Import necessary libraries

In [5]:
 # %pip install names


     -------------------------------------- 789.1/789.1 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for names: filename=names-0.3.0-py3-none-any.whl size=803682 sha256=e197cb3b2d20dc8291bb299ce039f9f0a188ed522c61e8f699ae0bc8b762192e
  Stored in directory: c:\users\sujat\appdata\local\pip\cache\wheels\f1\bc\04\55ab9499ea02359ece8b02b4169ebb30aa52d82b84c13fc506
Successfully built names


In [3]:
import numpy as np
import names


In [2]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['student_marks']
collection = db['marks']

#### Entering Data into MongoDB

In [5]:
for i in range(125):
    ch = np.random.randint(0,2)
    if ch == 0:
        name = names.get_first_name(gender='male')
        gender = 'M'
    elif ch == 1:
        name = names.get_first_name(gender='female')
        gender = 'F'    
    roll = f'0{i+1}'
    maths = np.random.randint(0, 100)
    english = np.random.randint(0, 100)
    physics = np.random.randint(0, 100)
    chemistry = np.random.randint(0, 100)
    biology = np.random.randint(0, 100)

    record = {
        'name': name,
        'roll': roll,
        'gender': gender,
        'maths': maths,
        'english': english,
        'physics': physics,
        'chemistry': chemistry,
        'biology': biology
    }
    collection.insert_one(record)

#### Testing Query

Populating Database

In [14]:
import names
for i in range(10):
    ch = np.random.randint(0,2)
    if ch == 0:
        name = names.get_first_name(gender='male')
        gender = 'M'
    elif ch == 1:
        name = names.get_first_name(gender='female')
        gender = 'F'
    roll = f'0{i+7}'
    maths = np.random.randint(0, 100)
    english = np.random.randint(0, 100)
    physics = np.random.randint(0, 100)
    chemistry = np.random.randint(0, 100)
    biology = np.random.randint(0, 100)

    record = {
        'name': name,
        'gender': gender, 
        'roll': roll,
        'maths': maths,
        'english': english,
        'physics': physics,
        'chemistry': chemistry,
        'biology': biology
    }

    print(record)

{'name': 'Maria', 'gender': 'F', 'roll': '07', 'maths': 57, 'english': 90, 'physics': 31, 'chemistry': 71, 'biology': 81}
{'name': 'Vincent', 'gender': 'M', 'roll': '08', 'maths': 54, 'english': 63, 'physics': 20, 'chemistry': 71, 'biology': 22}
{'name': 'John', 'gender': 'M', 'roll': '09', 'maths': 76, 'english': 1, 'physics': 87, 'chemistry': 81, 'biology': 9}
{'name': 'Michael', 'gender': 'M', 'roll': '010', 'maths': 99, 'english': 65, 'physics': 86, 'chemistry': 7, 'biology': 2}
{'name': 'Erica', 'gender': 'F', 'roll': '011', 'maths': 2, 'english': 54, 'physics': 7, 'chemistry': 39, 'biology': 62}
{'name': 'David', 'gender': 'M', 'roll': '012', 'maths': 85, 'english': 4, 'physics': 66, 'chemistry': 7, 'biology': 69}
{'name': 'Terry', 'gender': 'M', 'roll': '013', 'maths': 67, 'english': 85, 'physics': 51, 'chemistry': 71, 'biology': 74}
{'name': 'Laurence', 'gender': 'M', 'roll': '014', 'maths': 4, 'english': 18, 'physics': 34, 'chemistry': 33, 'biology': 55}
{'name': 'Amanda', 'ge

Editing record

In [11]:
name_1 = 'Ally'
roll_1 = '0126'
maths = 1
english = 2
physics = 3
chemistry = 4
biology = 5
collection.update_one(
    {'roll' : roll_1, 'name': name_1},
    {
        '$set' :{
            'maths': maths,
            'english': english,
            'physics': physics,
            'chemistry': chemistry,
            'biology': biology

        }
    }
)

Finding percentage

In [52]:
name_1 = 'Jason'
roll_1 = '04'
stud = list(collection.aggregate([
    {'$match':{'roll':roll_1, 'name':name_1}},
    {
        '$addFields':{
            'total_marks': {
                '$sum':['$maths','$english','$physics','$chemistry','$biology']
            }
        }
    },
    {
        '$addFields':{
            'total_percentage':{
                '$divide':['$total_marks',5]
            }

        }

    }
    ]))
stud_detail = {}
stud_detail = stud[0]
print(stud_detail['name'],stud_detail['total_percentage'])

Jason 39.8


Finding Percentile

In [143]:
name_1 = 'Trista'
roll_1 = '0121'
# Get the target student
target_student = collection.find_one({'name': name_1, 'roll': roll_1})

# Get the total number of students
total_students = collection.count_documents({})

# Sort the student records by total marks in descending order
pipeline = [
    {
        
        '$project': {
            'name': 1,
            'roll': 1,
            'total_marks': {
                '$sum': ['$maths', '$english', '$physics', '$chemistry', '$biology']
            }
        }
    },
    {
        '$sort': {'total_marks': -1}
    }
]
sorted_students = list(collection.aggregate(pipeline))
total_marks_list = [doc['total_marks'] for doc in sorted_students]

# Find the rank of the target student
for student in sorted_students:
    if student['_id'] == target_student['_id']:
        target_total_marks = student['total_marks']
        students_less_than_target = len([marks for marks in total_marks_list if marks <= target_total_marks])
        break

# Calculate the percentile
percentile = ( students_less_than_target / total_students) * 100

list_of_toppers = []
# Calculate class topper
for student in sorted_students:
    if student['total_marks'] == total_marks_list[0]:
        list_of_toppers.append(student)

 

In [150]:
print(list_of_toppers[0]['name'])

Ronnie


In [142]:
print(percentile)

96.8


In [136]:
121/125

0.968

In [117]:
print(list(sorted_students))

[{'_id': ObjectId('64738660c439dff35e578fb4'), 'name': 'Ronnie', 'roll': '05', 'total_marks': 340}, {'_id': ObjectId('64738660c439dff35e578fb9'), 'name': 'Kathy', 'roll': '010', 'total_marks': 340}, {'_id': ObjectId('64738660c439dff35e578fc4'), 'name': 'Christopher', 'roll': '021', 'total_marks': 340}, {'_id': ObjectId('64738660c439dff35e57902b'), 'name': 'Paul', 'roll': '0124', 'total_marks': 340}, {'_id': ObjectId('64738660c439dff35e579028'), 'name': 'Trista', 'roll': '0121', 'total_marks': 339}, {'_id': ObjectId('64738660c439dff35e578fda'), 'name': 'William', 'roll': '043', 'total_marks': 338}, {'_id': ObjectId('64738660c439dff35e578fef'), 'name': 'Maria', 'roll': '064', 'total_marks': 337}, {'_id': ObjectId('64738660c439dff35e57901a'), 'name': 'Anne', 'roll': '0107', 'total_marks': 334}, {'_id': ObjectId('64738660c439dff35e578fc1'), 'name': 'Nathaniel', 'roll': '018', 'total_marks': 330}, {'_id': ObjectId('64738660c439dff35e578ff7'), 'name': 'Michael', 'roll': '072', 'total_marks':

Deleting record

In [10]:
name_1 = 'Ally'
roll_1 = '0126'
collection.delete_one({'roll' : roll_1, 'name': name_1})